**https://www.youtube.com/watch?v=vF_qrJ0qHkg**

**https://colab.research.google.com/drive/1wfBW4ekWIkCpmL411x2mTg-UzE3nGJE_?usp=sharing#scrollTo=pPBMr4MuUgqs**

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [ ]:
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest

**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')[0:1000]
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')[0:1000]
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')[0:1000]

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((1000, 3), (1000, 3), (1000, 2))

**Text Preprocessing**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('omw-1.4')
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [ ]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [ ]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((1000, 4), (1000, 4), (1000, 3))

In [ ]:
sentences = []

for i in Train_data['cleanText']:
    sentences.append(i)

for j in Val_data['cleanText']:
    sentences.append(i)

for k in Test_data['cleanText']:
    sentences.append(i)

#**Word2Vec Embeddings**

**Install Spacy**

In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.4 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
x = 'king man woman'
doc = nlp(x)

for token1 in doc:
    for token2 in doc:
        print((token1.text, token2.text, token1.similarity(token2)))

('king', 'king', 1.0)
('king', 'man', 0.41661593317985535)
('king', 'woman', 0.35728728771209717)
('man', 'king', 0.41661593317985535)
('man', 'man', 1.0)
('man', 'woman', 0.8273443579673767)
('woman', 'king', 0.35728728771209717)
('woman', 'man', 0.8273443579673767)
('woman', 'woman', 1.0)


**Model Building**

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-2k4t8_vx
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-2k4t8_vx
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=3c0d0f635be55552391712e57eaf0f284c8a7b66a7df0f495365eed8653aae60
  Stored in directory: /tmp/pip-ephem-wheel-cache-5w0le9ty/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps

In [ ]:
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
x = 'you have @text dAta @#this youuuuuuu'
get_clean(x)

'you have text data this you'

**Word2Vec Conversion**

In [ ]:
x = 'cat dog'
doc = nlp(x)

In [ ]:
doc.vector.shape

(300,)

In [ ]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [ ]:
get_vec(x)

array([  2.4681    ,   4.24725   ,  -6.487     , -10.7215    ,
         0.92620105,   0.19215   ,  -3.8025    ,  -4.4794    ,
         0.12358999,   2.26012   ,   6.9578    ,   3.3215    ,
         0.25315368,   3.26315   ,   6.3065    ,  -5.63415   ,
        -0.1302725 ,  -3.80655   ,   1.7955    ,  -1.3794    ,
        -4.8231497 ,   4.03365   ,  -3.9047499 ,  -3.6059499 ,
        -0.51816   ,  -0.34360003,  -4.32535   ,   7.1096    ,
        -5.60805   ,  -1.57005   ,  -0.68139994,   0.06522998,
        -3.30515   ,   0.11580002,   0.46775   ,  -7.51595   ,
         1.401765  ,   4.31465   ,   4.9555    ,  -3.54565   ,
        -0.28759998,  -4.9586    ,   9.3208    ,   4.44065   ,
         0.802095  ,   0.130241  ,   2.8218    ,  -4.51365   ,
         0.28776652,  -2.70525   ,   3.55798   ,  12.496     ,
         3.9459999 ,   2.5117998 ,  -2.7475    ,   2.2166    ,
         4.8261    ,   0.75804996,   3.4015    ,   1.6985    ,
         0.599965  ,  -7.1053505 ,  -0.54585004,   2.40

In [ ]:
get_vec(x).shape

(300,)

**Getting word vectors for text**

In [ ]:
#Train data
Train_data['vec'] = Train_data['cleanText'].apply(lambda x: get_vec(x))

#Val data
Val_data['vec'] = Val_data['cleanText'].apply(lambda x: get_vec(x))

#Test data
Test_data['vec'] = Test_data['cleanText'].apply(lambda x: get_vec(x))

In [ ]:
Train_data.head(10)

,id,tweet,label,cleanText,vec
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...,"[-0.28593546, -0.3856088, -1.8129069, 1.000557..."
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...,"[-1.2702777, -0.13523456, -1.8308263, 2.349623..."
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...,"[0.12449169, 0.41355166, -1.2276736, 1.2217699..."
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...,"[1.1677426, 0.30753973, -0.064459294, 1.083999..."
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...,"[-1.4634335, 1.1837531, -3.254066, 1.9648373, ..."
5,6,"Covid Act Now found ""on average each person in...",real,covid act found average person illinois covid ...,"[0.9754168, 0.7205716, -2.4040794, 1.1087513, ..."
6,7,If you tested positive for #COVID19 and have n...,real,tested positive covid symptoms stay home away ...,"[1.1947285, 1.1936328, -2.4118092, 0.5734074, ..."
7,8,Obama Calls Trump’s Coronavirus Response A Cha...,fake,obama calls trump coronavirus response chaotic...,"[1.0938054, 0.05980151, -1.60661, 1.6324642, 2..."
8,9,"???Clearly, the Obama administration did not l...",fake,clearly obama administration leave kind game p...,"[-0.3091667, 1.5663044, -2.316999, -0.8252267,..."
9,10,Retraction—Hydroxychloroquine or chloroquine w...,fake,retraction hydroxychloroquine chloroquine with...,"[0.27890638, 0.16112436, -0.10680265, 0.873402..."


In [ ]:
Val_data.head(10)

,id,tweet,label,cleanText,vec
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...,"[-0.68882996, 0.2290033, -0.8924601, 0.8497011..."
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...,"[-1.5461936, 1.416388, -1.0224106, 1.1580822, ..."
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin,"[1.7226652, -0.10857717, -1.2261333, -0.248846..."
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...,"[-0.13681875, 0.027411707, -1.761468, 1.312199..."
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...,"[3.5766473, 0.14633845, 0.15774365, 1.8407121,..."
5,6,No one can leave managed isolation for any rea...,real,one leave managed isolation reason without ret...,"[-0.87937254, 1.3023043, -0.70043063, 0.440710..."
6,7,#IndiaFightsCorona India has one of the lowest...,real,indiafightscorona india one lowest covid morta...,"[-0.6618637, 0.12927827, -1.5173479, 1.3381639..."
7,8,RT @WHO: #COVID19 transmission occurs primaril...,real,covid transmission occurs primarily direct ind...,"[0.13368249, 0.14871477, -1.72161, 1.8268703, ..."
8,9,News and media outlet ABP Majha on the basis o...,fake,news media outlet abp majha basis internal mem...,"[-1.2051947, 0.16272691, -1.7717731, 0.6580345..."
9,10,???Church services can???t resume until we???r...,fake,church services resume vaccinated says bill gates,"[0.15197995, -0.22400427, -1.2113158, 0.405888..."


In [ ]:
Test_data.head(10)

,id,tweet,cleanText,vec
0,1,Our daily update is published. States reported...,daily update published states reported tests n...,"[0.2563394, -0.027521238, -0.66661173, 1.27986..."
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid,"[3.3735332, 1.4379058, -0.93348664, 0.52401, -..."
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...,"[1.13105, 2.9406714, -1.8205101, 0.99381113, 1..."
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...,"[0.07514732, 0.34164032, -1.0331218, 1.1537108..."
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...,"[0.19960016, 0.38874912, -0.7893367, 1.3366054..."
5,6,Low #vitaminD was an independent predictor of ...,low vitamind independent predictor worse progn...,"[0.21843374, 1.3041446, -0.76450247, 1.5978024..."
6,7,A common question: why are the cumulative outc...,common question cumulative outcome numbers sma...,"[-1.054263, -0.26391044, -1.3295479, 1.909098,..."
7,8,The government should consider bringing in any...,government consider bringing new national lock...,"[-0.26984155, -0.18456088, -0.31638825, 0.1580..."
8,9,Our daily update is published. We’ve now track...,daily update published tracked million tests y...,"[-0.28771433, -0.24244763, 0.00262175, 2.13439..."
9,10,Breakdown of testing: 4 air crew 97 hotel &amp...,breakdown testing air crew hotel amp health st...,"[-0.022250745, 0.9279498, -1.6946707, 0.845794..."


**Splitting Dataset**

In [ ]:
X_train = Train_data['vec'].to_numpy()
X_train = X_train.reshape(-1, 1)

X_test = Val_data['vec'].to_numpy()
X_test = X_test.reshape(-1, 1)

In [ ]:
X_train.shape, X_test.shape

((1000, 1), (1000, 1))

In [ ]:
X_train = np.concatenate(np.concatenate(X_train, axis = 0), axis = 0).reshape(-1, 300)
X_test = np.concatenate(np.concatenate(X_test, axis = 0), axis = 0).reshape(-1, 300)

In [ ]:
X_train.shape, X_test.shape

((1000, 300), (1000, 300))

In [ ]:
y_train = Train_data['label']
y_test = Val_data['label']

**Model Evaluation**

In [ ]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    #"MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gaussian Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [ ]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [ ]:
def batch_classify(dict_classifiers, X_train,X_test,y_train,y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        y_pred = classifier.predict(X_test)
        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [ ]:
dict_super_models = batch_classify(dict_classifiers, X_train,X_test,y_train,y_test)

Nearest Neighbors
Accuracy: 0.840000
Precision: 0.846343
Recall: 0.840000
F1 score: 0.838409
Cohens kappa: 0.676130
[[354 117]
 [ 43 486]]
train score:{0.93} test_score:{0.84}
LogisticRegression
Accuracy: 0.871000
Precision: 0.871825
Recall: 0.871000
F1 score: 0.870672
Cohens kappa: 0.740252
[[392  79]
 [ 50 479]]
train score:{0.91} test_score:{0.87}
Decision Tree
Accuracy: 0.716000
Precision: 0.717824
Recall: 0.716000
F1 score: 0.713527
Cohens kappa: 0.425536
[[295 176]
 [108 421]]
train score:{0.91} test_score:{0.72}
Random Forest
Accuracy: 0.752000
Precision: 0.760110
Recall: 0.752000
F1 score: 0.747846
Cohens kappa: 0.496346
[[296 175]
 [ 73 456]]
train score:{0.85} test_score:{0.75}
MLP
Accuracy: 0.871000
Precision: 0.872504
Recall: 0.871000
F1 score: 0.870542
Cohens kappa: 0.740010
[[388  83]
 [ 46 483]]
train score:{0.96} test_score:{0.87}
AdaBoost
Accuracy: 0.816000
Precision: 0.816079
Recall: 0.816000
F1 score: 0.815711
Cohens kappa: 0.629896
[[369 102]
 [ 82 447]]
train score

In [ ]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [ ]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
12,Gaussian Process Classifier,0.884,0.884179,0.884,0.883862,0.766783
1,LogisticRegression,0.871,0.871825,0.871,0.870672,0.740252
4,MLP,0.871,0.872504,0.871,0.870542,0.740010
11,LGBM,0.858,0.857960,0.858,0.857925,0.714776
9,GradientBoost,0.852,0.851954,0.852,0.851963,0.702863
10,SGD,0.848,0.848231,0.848,0.848061,0.695258
13,Support Vector Machine 1,0.843,0.843407,0.843,0.842672,0.684021
0,Nearest Neighbors,0.840,0.846343,0.840,0.838409,0.676130
7,LinearDiscriminant,0.823,0.825860,0.823,0.821886,0.642517
5,AdaBoost,0.816,0.816079,0.816,0.815711,0.629896


In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &  Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &     0.840 &   0.846343 &   0.840 &  0.838409 &  0.676130 \\
          LogisticRegression &     0.871 &   0.871825 &   0.871 &  0.870672 &  0.740252 \\
               Decision Tree &     0.716 &   0.717824 &   0.716 &  0.713527 &  0.425536 \\
               Random Forest &     0.752 &   0.760110 &   0.752 &  0.747846 &  0.496346 \\
                         MLP &     0.871 &   0.872504 &   0.871 &  0.870542 &  0.740010 \\
                    AdaBoost &     0.816 &   0.816079 &   0.816 &  0.815711 &  0.629896 \\
                  GaussianNB &     0.773 &   0.776008 &   0.773 &  0.771157 &  0.540995 \\
          LinearDiscriminant &     0.823 &   0.825860 &   0.823 &  0.821886 &  0.642517 \\
       QuadraticDiscriminant &     0.801 &   0.845726 &   0.801 &  0.796869 &  0.608628 \\
               GradientBoost &     0.852 &   0.8